In [1]:
import pickle
import sys
import pandas as pd

In [2]:
data = pickle.load(open('student_answer.pkl','rb'))
hint_data = pickle.load(open('student_hint.pkl','rb'))

attempt = {}
for line in data:
    if line['timestamp'] == 'null':
        continue
    #if line['attempt'] == 'no attempt':
    #    continue
    answer = [line['timestamp'],line['attempt'],line['answer'], line['score']]
    key = (line['set_id'],line['problem_id'],line['part_id'])
    if key not in attempt:
        attempt[key] = {}
    student_key = (str(line['user_id']).replace('L',''),line['username'])
    if student_key not in attempt[key]:
        attempt[key][student_key] = []
    attempt[key][student_key].append(answer)

In [9]:
### Run this to see the number of wrong attempts for each week each problem and each part.

### Type in integer for the week id
week_id = '4'

attempt_count_data = []
for problem in attempt:
    if problem[0] == week_id:
        total_attempt_number = 0
        for student in attempt[problem]:
            for ans in attempt[problem][student]:
                if ans[3] == '0':
                    total_attempt_number += 1
                elif ans[3] == '1' or ans[3] == '2':
                    break
        attempt_count_data.append([problem[0], problem[1], problem[2], str(total_attempt_number)])

In [10]:
### Sort the count by attempts
attempt_count_data = sorted(attempt_count_data, key=lambda x: (int(x[1]), int(x[2])))
ind = [a[0] for a in attempt_count_data]
ind2 = [a[1] for a in attempt_count_data]
ind3 = [a[2] for a in attempt_count_data]
d = [a[3] for a in attempt_count_data]
col = ['Total Attempts']
df = pd.DataFrame(d, index = [ind,ind2, ind3], columns = col)
df.index.names = ['Week ID', 'Problem ID', 'Part ID']
df

Total Attempts
Week ID Problem ID Part ID               
4       1          1                  167
                   2                  304
                   3                  180
        2          1                  210
                   2                   76
                   3                   70
                   4                  116
                   5                  105
                   6                  170
                   7                  247
        3          1                   77
                   2                  197
                   3                  203
                   4                  294
        4          1                  101
                   2                   89
                   3                   78
        5          1                  439
                   2                  956
                   3                   67
                   4                  466
        6          1                  249
                   2                  243
        7          1                  581
                   2                  245
                   3                  333
        8          1                  154
                   2                  201
                   3                  716
        9          1                  328
                   2                  214
                   3                  246
                   4                   35
                   5                  183
                   6                  254

In [11]:
### Sort the count by problem id and part id
attempt_count_data = sorted(attempt_count_data, key=lambda x: int(x[3]), reverse = True)
col = ['Week ID', 'Problem ID', 'Part ID', 'Total Attempts']
pd.DataFrame(attempt_count_data, columns = col)

,Week ID,Problem ID,Part ID,Total Attempts
0,4,5,2,956
1,4,8,3,716
2,4,7,1,581
3,4,5,4,466
4,4,5,1,439
5,4,7,3,333
6,4,9,1,328
7,4,1,2,304
8,4,3,4,294
9,4,9,6,254


In [41]:
## Yoav: Generate a single big dataframe for the given week. Generate the smaller reports by selecting from that df.

### Select certain week_id, problem_id, and part_id as int to show below

### Type in integer for the week id
week_id = 4

### If you want to see all problems of certain week, put problem_usr = 0
problem_id = 5

### If you want to see all parts of certain problem,  put part_usr = 0
part_id = 2

In [42]:
### Run this to output the student attempts and answers.
attempt_data = []
ind0 = []
ind1 = []
ind2 = []
col = ["Attempt", "Answer", "Correctness"]
for problem in attempt:
    if (week_id != 0 and problem[0] != str(week_id)) or \
    (problem_id != 0 and problem[1] != str(problem_id)) or \
    (part_id != 0 and problem[2] != str(part_id)):
        continue 
    print "Data for Week {0}, Problem {1}, Part {2}.".format(problem[0], problem[1], problem[2])
    for student in sorted(attempt[problem],key=lambda x:int(x[0])):
        previous = []
        for ans in attempt[problem][student][:10]:
            if ans[1] == 'null' or ans[1] == '' or ans[2] == 'null':
                continue
            if ans[1:] == previous:
                continue
            ind0.append(student[0])
            ind1.append(student[1])
            ind2.append(ans[0])
            attempt_data.append(ans[1:])
            previous = ans[1:]
ans_df = pd.DataFrame(attempt_data, index = [ind0,ind1,ind2], columns = col)
ans_df.index.names = ['ID', 'Name','TimeStamp']
ans_df

Data for Week 4, Problem 5, Part 2.


Attempt  \
ID  Name           TimeStamp                                        
31  yazin-yousif   2016-10-19 04:39:44                        178   
32  abtom          2016-10-21 06:02:02        C(47, 2) - C(43, 2)   
                   2016-10-21 06:09:40         C(47,2) - C(43, 2)   
33  nesengup       2016-10-21 16:25:45                        178   
34  mvkrishn       2016-10-17 23:36:56                       4*46   
                   2016-10-17 23:44:54                   4 * 46/2   
                   2016-10-17 23:46:19          C(47,2) - C(43,2)   
35  adejesus       2016-10-20 02:58:11                  C(47,2)     
                   2016-10-20 03:38:14   C(4,1)*C(46,1) - C(47,2)   
                   2016-10-20 03:38:48   C(46,2) - C(4,1)*C(46,1)   
                   2016-10-20 06:30:11          C(47,2) - C(39,2)   
                   2016-10-20 06:33:41          C(47,2) - C(39,1)   
                   2016-10-20 06:44:24          C(47,2) - C(43,1)   
                   2016-10-20 06:44:46          C(47,2) - C(43,2)   
37  aliceh         2016-10-21 05:02:51                       13*4   
                   2016-10-21 05:04:37                    13*4*46   
                   2016-10-21 05:05:03                  4*C(46,1)   
                   2016-10-21 05:06:39                       4*46   
                   2016-10-21 05:07:46            c(47,4)*C(46,1)   
                   2016-10-21 05:08:55                      13*16   
                   2016-10-21 05:14:47        C(47,2)-(12*11*4*4)   
                   2016-10-21 05:15:16       C(47,2)-(C(12,2)*16)   
                   2016-10-21 05:16:11       C(47,2)-(C(43,2)*16)   
38  NicoleStone    2016-10-16 04:19:48                          4   
                   2016-10-16 04:24:58                       4*47   
                   2016-10-16 04:26:49                       4*46   
                   2016-10-21 18:43:16                          4   
                   2016-10-21 18:45:15                       4*46   
                   2016-10-21 18:52:48            C(47,2)-C(43,2)   
39  JunnelReboquio 2016-10-21 14:01:19              C(4,1)*C(5,1)   
...                                                           ...   
343 RyanJHill      2016-10-21 17:47:42                 1-C(47,43)   
                   2016-10-21 18:03:19                   1-C(7,4)   
                   2016-10-21 18:08:31                 1-C(47,43)   
                   2016-10-21 18:09:51                  1-C(43,2)   
345 wesleymwong    2016-10-21 08:28:44  1.0*C(52-5,2)-C(52-5-4,2)   
348 gurkiratsingh  2016-10-21 09:45:42        1 * 4 * 13 * 4 - 1    
                   2016-10-21 09:50:11                     4*13*4   
                   2016-10-21 09:57:01  C(4,1) + C(48,1) + C(4,2)   
                   2016-10-21 09:58:12  C(4,1) * C(48,1) + C(4,2)   
                   2016-10-21 10:04:04          C(52,2) - C(47,2)   
                   2016-10-21 10:06:00          C(52,2) - C(48,2)   
                   2016-10-21 10:07:22                        198   
                   2016-10-21 10:12:50      (C(52,2) - C(48,2))/2   
                   2016-10-21 10:17:10                    C(43,2)   
                   2016-10-21 10:19:28           C(52,2) - (43,2)   
349 DarrenYang     2016-10-17 02:05:00                47*46-44*43   
                   2016-10-17 02:06:34                47*46-43*42   
                   2016-10-17 02:08:57            47*46/2-43*42/2   
351 nicoleyfq      2016-10-19 07:52:45               C(47,4)*46/2   
                   2016-10-19 08:03:56            C(47,2)-C(43,2)   
352 whcombs        2016-10-21 21:27:49                          4   
                   2016-10-21 21:29:22                        4*4   
                   2016-10-21 21:32:29                        4*3   
                   2016-10-21 21:41:35          4*(47-4)+(47-4)*4   
                   2016-10-21 21:45:14                        4*3   
                   2016-10-21 21:47:55                  

In [45]:
### Run this to see students' answers with hints.

#combine two dic
def gen_ans_df():
    hint_attempt = {}
    for problem in attempt:
        hint_attempt[problem] = {}
        for student in attempt[problem]:
            hint_attempt[problem][student] = attempt[problem][student][:]
            if problem in hint_data and student in hint_data[problem]:
                for hint_content in hint_data[problem][student]:
                    if hint_content[1] != "no attempt":
                        hint_attempt[problem][student].append(hint_content)
                hint_attempt[problem][student].sort()

    hint_attempt_data = []
    hint_ind0 = []
    hint_ind1 = []
    hint_ind2 = []
    col = ["Attempt", "Attempt/Hint", "Correctness"]
    for problem in hint_attempt:
        if (week_id != 0 and problem[0] != str(week_id)) or \
        (problem_id != 0 and problem[1] != str(problem_id)) or \
        (part_id != 0 and problem[2] != str(part_id)):
            continue 
        print "Data for Week {0}, Problem {1}, Part {2}.".format(problem[0], problem[1], problem[2])
        for student in sorted(hint_attempt[problem], key=lambda x:int(x[0])):
            previous = []
            for ans in hint_attempt[problem][student]:
                if ans[1] == 'null' or ans[1] == '' or ans[2] == 'null':
                    continue
                if ans[1:] == previous:
                    continue
                hint_ind0.append(student[0])
                hint_ind1.append(student[1])
                hint_ind2.append(ans[0])
                hint_attempt_data.append(ans[1:])
                previous = ans[1:]
    ans_df = pd.DataFrame(hint_attempt_data, index = [hint_ind0, hint_ind1, hint_ind2], columns = col)
    ans_df.index.names = ['ID', 'Name','TimeStamp']
    return ans_df
gen_ans_df()

Data for Week 4, Problem 5, Part 2.


Attempt  \
ID  Name          TimeStamp                                        
31  yazin-yousif  2016-10-19 04:39:44                        178   
32  abtom         2016-10-21 06:02:02        C(47, 2) - C(43, 2)   
                  2016-10-21 06:09:40         C(47,2) - C(43, 2)   
33  nesengup      2016-10-21 16:25:45                        178   
34  mvkrishn      2016-10-17 23:36:56                       4*46   
                  2016-10-17 23:44:54                   4 * 46/2   
                  2016-10-17 23:46:19          C(47,2) - C(43,2)   
35  adejesus      2016-10-20 02:58:11                  C(47,2)     
                  2016-10-20 03:38:14   C(4,1)*C(46,1) - C(47,2)   
                  2016-10-20 03:38:48   C(46,2) - C(4,1)*C(46,1)   
                  2016-10-20 06:30:11          C(47,2) - C(39,2)   
                  2016-10-20 06:33:41          C(47,2) - C(39,1)   
                  2016-10-20 06:44:24          C(47,2) - C(43,1)   
                  2016-10-20 06:44:46          C(47,2) - C(43,2)   
37  aliceh        2016-10-21 05:02:51                       13*4   
                  2016-10-21 05:04:37                    13*4*46   
                  2016-10-21 05:05:03                  4*C(46,1)   
                  2016-10-21 05:06:39                       4*46   
                  2016-10-21 05:07:46            c(47,4)*C(46,1)   
                  2016-10-21 05:08:55                      13*16   
                  2016-10-21 05:14:47        C(47,2)-(12*11*4*4)   
                  2016-10-21 05:15:16       C(47,2)-(C(12,2)*16)   
                  2016-10-21 05:16:11       C(47,2)-(C(43,2)*16)   
                  2016-10-21 05:16:33   C(47,2)-(C(43,2)*C(4,2))   
                  2016-10-21 05:17:51  C(47,2)-(C(43,1)*C(42,1))   
                  2016-10-21 05:21:00          C(47,2)-(C(43,2))   
38  NicoleStone   2016-10-16 04:19:48                          4   
                  2016-10-16 04:24:58                       4*47   
                  2016-10-16 04:26:49                       4*46   
                  2016-10-21 18:43:16                          4   
...                                                          ...   
348 gurkiratsingh 2016-10-21 10:17:10                    C(43,2)   
                  2016-10-21 10:19:28           C(52,2) - (43,2)   
                  2016-10-21 10:20:06          C(47,2) - C(43,2)   
349 DarrenYang    2016-10-17 02:05:00                47*46-44*43   
                  2016-10-17 02:06:34                47*46-43*42   
                  2016-10-17 02:08:57            47*46/2-43*42/2   
351 nicoleyfq     2016-10-19 07:52:45               C(47,4)*46/2   
                  2016-10-19 08:03:56            C(47,2)-C(43,2)   
352 whcombs       2016-10-21 21:27:49                          4   
                  2016-10-21 21:29:22                        4*4   
                  2016-10-21 21:32:29                        4*3   
                  2016-10-21 21:41:35          4*(47-4)+(47-4)*4   
                  2016-10-21 21:45:14                        4*3   
                  2016-10-21 21:47:55                       4*43   
                  2016-10-21 21:52:44                        4*3   
                  2016-10-21 21:54:36              4*3+4*43+43*4   
                  2016-10-21 22:05:51                         12   
                  2016-10-21 22:06:34                          4   
                  2016-10-21 22:09:48                          6   
                  2016-10-21 22:12:32                        4*3   
                  2016-10-21 22:14:48                     C(4,2)   
                  2016-10-21 22:17:53                   43*4+4*3   
                  2016-10-21 22:21:53                   4*3+4*12   
                  2016-10-21 22:31:43            C(47,2)-(12*12)   
                  2016-10-21 22:42:15                   4*3+4*12   
                  2016-10-21 22:44:47                   4*3+4*43   
                  2016-10-21 22:54:34        

In [48]:
problem=[4,8,3]
ans_df=gen_ans_df()
selector=[ a is None for a in list(ans_df['Correctness'])] 
ans_df[selector]

Data for Week 4, Problem 5, Part 2.


Attempt  \
ID  Name          TimeStamp                                           
79  DongyoonHan   2016-10-21 08:35:25                           184   
123 msp           2016-10-21 15:44:49  4*(C(47,1)-(C(43,1)*C(4,1)))   
145 gtran         2016-10-21 16:52:17                           184   
156 kevinmc       2016-10-21 10:56:43                             4   
226 zkvick        2016-10-20 21:49:42                             4   
                  2016-10-20 21:56:44               C(47,2)-(43*42)   
                  2016-10-20 21:57:24              C(47,2)-C(47,43)   
271 WilliamZhu    2016-10-21 07:05:05                             4   
278 PhillipJo     2016-10-21 08:21:55                           140   
297 Brnkvu        2016-10-20 23:42:52                            12   
335 sokunev       2016-10-21 09:14:20       1 - ((12/13) * (12/13))   
339 jarninfang    2016-10-21 07:03:23                          8/52   
348 gurkiratsingh 2016-10-21 10:07:40                           198   

                                                                            Attempt/Hint  \
ID  Name          TimeStamp                                                                
79  DongyoonHan   2016-10-21 08:35:25  Please write an expression, not the final nume...   
123 msp           2016-10-21 15:44:49                        Can the answer be negative?   
145 gtran         2016-10-21 16:52:17  Please write an expression, not the final nume...   
156 kevinmc       2016-10-21 10:56:43  Please write an expression, not the final nume...   
226 zkvick        2016-10-20 21:49:42  Please write an expression, not the final nume...   
                  2016-10-20 21:56:44                        Can the answer be negative?   
                  2016-10-20 21:57:24                        Can the answer be negative?   
271 WilliamZhu    2016-10-21 07:05:05  Please write an expression, not the final nume...   
278 PhillipJo     2016-10-21 08:21:55  Please write an expression, not the final nume...   
297 Brnkvu        2016-10-20 23:42:52  Please write an expression, not the final nume...   
335 sokunev       2016-10-21 09:14:20                      Can the answer be fractional?   
339 jarninfang    2016-10-21 07:03:23                      Can the answer be fractional?   
348 gurkiratsingh 2016-10-21 10:07:40  Please write an expression, not the final nume...   

                                      Correctness  
ID  Name          TimeStamp                        
79  DongyoonHan   2016-10-21 08:35:25        None  
123 msp           2016-10-21 15:44:49        None  
145 gtran         2016-10-21 16:52:17        None  
156 kevinmc       2016-10-21 10:56:43        None  
226 zkvick        2016-10-20 21:49:42        None  
                  2016-10-20 21:56:44        None  
                  2016-10-20 21:57:24        None  
271 WilliamZhu    2016-10-21 07:05:05        None  
278 PhillipJo     2016-10-21 08:21:55        None  
297 Brnkvu        2016-10-20 23:42:52        None  
335 sokunev       2016-10-21 09:14:20        None  
339 jarninfang    2016-10-21 07:03:23        None  
348 gurkiratsingh 2016-10-21 10:07:40        None